In [561]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [562]:
RANDOM_SEED = 42
NUMBER_OF_CATEGORIZED_WORDS=2000

# Wstępne przetwarzenie danych

## Wczytywanie danych

In [563]:
df = pd.read_csv("data/file.csv", index_col=0)

In [564]:
df.head()

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


## Przetwarzanie zbioru

In [565]:
stopwords  = stopwords.words("english")

In [566]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [567]:
stemmer = nltk.stem.PorterStemmer()

In [568]:

def preprocessTweets(t):
    # usuwa url
    preprocessedTweet = re.sub('(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', ' ', t) 
    # usuwa \\n
    preprocessedTweet = re.sub(r'\\n', ' ', preprocessedTweet) 
    # tylko litery - usuwa emotikony 
    preprocessedTweet = re.sub(r"[^a-zA-Z']", ' ', preprocessedTweet) 
    # male litery
    preprocessedTweet = preprocessedTweet.lower() 

    preprocessedTweet = preprocessedTweet.split()

    #usuwa zbedne slowa
    preprocessedTweet = [ word for word in preprocessedTweet if word not in stopwords]

    #dokonaj "stemming" - zamienia słowo na jego rdzeń
    preprocessedTweet = [ stemmer.stem(word) for word in preprocessedTweet]

    #sklej z powrotem
    return " ".join(preprocessedTweet)

In [569]:
df.tweets = df.tweets.apply(preprocessTweets)

In [570]:
df.iloc[19][0]

'ask chatgpt new ai system optim dialogu teach seo minut result'

In [571]:
df = shuffle(df, random_state=RANDOM_SEED)

## Podział danych

In [572]:
dataset_len = df.shape[0]
train_dataset_len = int(dataset_len*0.75)
test_dataset_len = int(dataset_len*0.25*0.5)

In [573]:
X_train, y_train = df.iloc[:train_dataset_len, 0], df.iloc[:train_dataset_len,1]
X_valid, y_valid = df.iloc[train_dataset_len:train_dataset_len+test_dataset_len, 0], df.iloc[train_dataset_len:train_dataset_len+test_dataset_len,1]
X_test, y_test = df.iloc[train_dataset_len+test_dataset_len:, 0], df.iloc[train_dataset_len + test_dataset_len:, 1]

## Przetwarzanie zbioru pod Bayens i Logistic Regression

In [574]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 

cv = TfidfVectorizer(binary=True, max_features=NUMBER_OF_CATEGORIZED_WORDS)
cv.fit(df.tweets)

X_vectorized_train = cv.transform(X_train)
X_vectorized_valid = cv.transform(X_valid)
X_vectorized_test = cv.transform(X_test)

In [575]:
X_vectorized_test.shape

(27413, 2000)

## Przetwarzanie zbioru pod RNN

In [576]:
words = tf.constant(cv.get_feature_names_out())
word_ids = tf.range(len(cv.get_feature_names_out()), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [577]:
X_train_tokenized = table.lookup(tf.strings.split(X_train)).to_tensor()

In [578]:
y_train_tokenized = tf.convert_to_tensor(np.where(y_train=="good", 2, np.where(y_train=="neutral", 1, 0)), dtype=tf.int64)

In [579]:
X_valid_tokenized = table.lookup(tf.strings.split(X_valid)).to_tensor()

In [580]:
y_valid_tokenized = tf.convert_to_tensor(np.where(y_valid=="good", 2, np.where(y_valid=="neutral", 1, 0)),dtype=tf.int64)

In [581]:
X_test_tokenized = table.lookup(tf.strings.split(X_test)).to_tensor()

In [582]:
y_test_tokenized = tf.convert_to_tensor(np.where(y_test=="good", 2, np.where(y_test=="neutral", 1, 0)), dtype=tf.int64)

# Bayens 

In [583]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_vectorized_train.toarray(), y_train);

In [584]:
accuracy_score(gnb.predict(X_vectorized_test.toarray()), y_test)

0.6248860029912815

# Logistic regression

In [585]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver="liblinear", random_state=RANDOM_SEED, C=5, penalty="l2", max_iter=1000)
model = lr.fit(X_vectorized_train,y_train)

In [586]:
accuracy_score(lr.predict(X_vectorized_test), y_test)

0.8037792288330354

# RNN

In [587]:


embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(NUMBER_OF_CATEGORIZED_WORDS + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(3, activation="sigmoid")
])
model.compile(loss="SparseCategoricalCrossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(x=X_train_tokenized, y=y_train_tokenized, epochs=5)



Epoch 1/5
5140/5140 [==============================] - 256s 49ms/step - loss: 0.4718 - accuracy: 0.8189
Epoch 2/5
5140/5140 [==============================] - 252s 49ms/step - loss: 0.3852 - accuracy: 0.8559
Epoch 3/5
5140/5140 [==============================] - 252s 49ms/step - loss: 0.3533 - accuracy: 0.8678
Epoch 4/5
5140/5140 [==============================] - 254s 49ms/step - loss: 0.3213 - accuracy: 0.8799
Epoch 5/5
5140/5140 [==============================] - 256s 50ms/step - loss: 0.2864 - accuracy: 0.8937


# BERT